<center>

<h1> Cut </h1>
<h2> CS3100 Fall 2019 </h2>
</center>

## Review

### Previously

* Generate and Test: A design pattern for logic programming

### This lecture

* Cuts
  + A mechanism for pruning Prolog search trees
  + Red and Green cuts

## Evaluator

Consider a simple evaluator for arithmetic expressions.

In [1]:
eval(plus(A,B),C) :- eval(A,VA), eval(B,VB), C is VA + VB.
eval(mult(A,B),C) :- eval(A,VA), eval(B,VB), C is VA * VB.
eval(A,A).

Added 3 clauses(s).

## Evaluator

What is the result of evaluating `1 + (4 * 5)`?

In [2]:
?- eval(plus(1,mult(4,5)),X) {1}.

X = 21 .

In [3]:
?- eval(plus(1,mult(4,5)),X) {2}.

: 

Trace `eval(plus(1,mult(4,5)),X)` by hand.

## Fixing the evaluator - with wrapper

Wrap the values in a function `value`.

In [4]:
eval2(plus(A,B),C) :- eval2(A,VA), eval2(B,VB), C is VA + VB.
eval2(mult(A,B),C) :- eval2(A,VA), eval2(B,VB), C is VA * VB.
eval2(value(A),A).

Added 3 clauses(s).

In [5]:
?- eval2(plus(value(1),mult(value(4),value(5))),X).

X = 21 .

## Fixing the evaluator - with cut

* The **cut** (!) is an extra-logical (outside pure logic) operator that prunes the search trees. 
* When the evaluation cross a cut, it prunes
  + All the subsequent possible branches in the parent.
  + All the subsequent possible branches in the preceeding sub-goals.

In [6]:
eval3(plus(A,B),C) :- !, eval3(A,VA), eval3(B,VB), C is VA + VB.
eval3(mult(A,B),C) :- !, eval3(A,VA), eval3(B,VB), C is VA * VB.
eval3(A,A).

Added 3 clauses(s).

In [7]:
?- eval3(plus(1,mult(4,5)),X).

X = 21 .

## Cut behaviour

In [8]:
p(a).
p(b).
r(c).
q(X) :- p(X), !.
q(X) :- r(X).

Added 5 clauses(s).

In [9]:
?- q(X).

X = a .

## Quiz

What does `split/3` do?

```prolog
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).
```

It splits the given list into elements less than 5 and greater than or equal to 5.

## Split

In [10]:
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).

Added 3 clauses(s).

In [11]:
?- split([1,2,3,4,5,6,7,8,9],L,R).

R = [ 5, 6, 7, 8, 9 ], L = [ 1, 2, 3, 4 ] .

* Observe that the last two cases are mutually exclusive. 
  + But Prolog still searches through the third rule, if second rule was successfully matched.

## Split with cut

In [ ]:
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, !, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, !, split(T,L,R).

* The second `!` is unnecessary as there are no further choices.
  + In fact, the predicate `H >= 5` is unnecessary since the only way to end up here is if the first rule failed.
  + But better to leave it there for readability.
* Recommendation:
  + Use cut to optimise execution, but retain predicates which help readability. 

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$. **✓**
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,!,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,!,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.  **✓**
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Red and Green cuts

```prolog
p :- a,!,b.
p :- c.
```

Since the cut above changes the logical meaning of the program, it is known as **Red cut**.

```prolog
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, !, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).
```

The cut in split does not change the logical meaning of the program. Hence, it is called **Green cut**.

<center>

<h1> Fin. </h1>
</center>